---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [1]:
def answer_one():
    import pandas as pd
    import numpy as np    
    cols = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    ScimEn = pd.read_excel('scimagojr-3.xlsx', na_values = np.NaN)    
    energy = pd.read_excel('Energy Indicators.xls', skipfooter = 38,skiprows=17, usecols=(2,3,4,5), names=('Country','Energy Supply','Energy Supply per Capita','% Renewable'), na_values = np.NaN)
    energy['Energy Supply'] = pd.to_numeric(energy['Energy Supply'], errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply'] * 1000000
    energy['Energy Supply per Capita'] = pd.to_numeric(energy['Energy Supply per Capita'], errors='coerce')
    energy['Country'].replace({ r'^([a-zA-Z\s,]+)(.*)$' : r'\1'}, regex=True, inplace = True)
    energy['Country'].replace({"Republic of Korea": "South Korea", "United States of America": "United States","United Kingdom of Great Britain and Northern Ireland": "United Kingdom","China, Hong Kong Special Administrative Region": "Hong Kong"}, inplace=True)
    energy['Country'] = energy['Country'].str.strip()    
    
    GDP = pd.read_csv('world_bank.csv',skiprows=4, na_values = np.NaN, usecols=[0, *range(50,60)])
    GDP.columns = ['Country' if c=='Country Name' else c for c in GDP.columns]
    GDP['Country'].replace({"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran","Hong Kong SAR, China": "Hong Kong"}, inplace=True)    
    GDP['Country'] = GDP['Country'].str.strip()
    
    en_gdp = pd.merge(energy, GDP, how='outer', left_on='Country', right_on='Country')
    merged = pd.merge(en_gdp,ScimEn, how='inner', left_on='Country', right_on = 'Country')
    merged = merged.set_index('Country')

    return merged[cols].sort_values(by='Rank')[:15]

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [ ]:
def answer_two():
    return merged.shape[0] - 15

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [ ]:
def answer_three():
    top = answer_one()
    col= ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    top['avgGDP'] = top[col].mean(axis=1,skipna=True)
    return top['avgGDP'].sort_values(ascending=False)

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [ ]:
def answer_four():
    top = answer_one()
    sorted_gdp = top.sort_values(by='2015', ascending=False)
    return sorted_gdp.iloc[5]['2015'] - sorted_gdp.iloc[5]['2006']

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [ ]:
def answer_five():
    top = answer_one()
    return top['Energy Supply per Capita'].mean(skipna=True)

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [ ]:
def answer_six():
    import pandas as pd
    import numpy as np    
    ScimEn = pd.read_excel('scimagojr-3.xlsx', na_values = np.NaN)
    top = answer_one()
    sort = top.sort_values(by='% Renewable', ascending=False)
    cnt = ScimEn[ScimEn['Rank'] == sort.iloc[0]['Rank']]
    return (cnt['Country'].iloc[0], sort.iloc[0]['% Renewable'])

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [ ]:
def answer_seven():
    import pandas as pd
    import numpy as np    
    ScimEn = pd.read_excel('scimagojr-3.xlsx', na_values = np.NaN)
    top = answer_one()
    top['ratio'] = top['Self-citations'] / top['Citations']
    sort = top.sort_values(by='ratio', ascending=False)
    cnt = ScimEn[ScimEn['Rank'] == sort.iloc[0]['Rank']]
    return (cnt['Country'].iloc[0], sort.iloc[0]['ratio'])

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [ ]:
def answer_eight():
    import pandas as pd
    import numpy as np    
    ScimEn = pd.read_excel('scimagojr-3.xlsx', na_values = np.NaN)
    top = answer_one()
    top['pop'] = top['Energy Supply'] / top['Energy Supply per Capita']
    sort = top.sort_values(by='pop', ascending=False)
    cnt = ScimEn[ScimEn['Rank'] == sort.iloc[2]['Rank']]
    return cnt['Country'].iloc[0]

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [ ]:
def answer_nine():
    top = answer_one()
    top['PopEst'] = top['Energy Supply'] / top['Energy Supply per Capita']
    top['cdpc'] = top['Citable documents'] / top['PopEst']
    return top.corr()['cdpc']['Energy Supply per Capita']

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [ ]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_ten():
    Top15 = answer_one()
    return "ANSWER"

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [ ]:
def answer_eleven():
    import pandas as pd
    import numpy as np    
    ScimEn = pd.read_excel('scimagojr-3.xlsx', na_values = np.NaN)
    top = answer_one()
    top['pop'] = top['Energy Supply'] / top['Energy Supply per Capita']
    ContinentDict  = {'China':'Asia', 
                      'United States':'North America', 
                      'Japan':'Asia', 
                      'United Kingdom':'Europe', 
                      'Russian Federation':'Europe', 
                      'Canada':'North America', 
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}
    merged = pd.merge(top, ScimEn, how='inner', left_on='Rank', right_on = 'Rank')
    merged['cont'] = [ContinentDict[c] for c in merged['Country']]
    cur = merged[['cont', 'pop']]
    grouped = cur.groupby(by='cont').count()
    grouped['sum'] = cur.groupby(by='cont').sum()['pop']
    grouped['mean'] = cur.groupby(by='cont').mean()['pop']
    grouped['std'] = cur.groupby(by='cont').std()['pop']
    grouped.columns = ['size' if c=='pop' else c for c in grouped.columns]
    return grouped

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    import pandas as pd
    import numpy as np    
    ScimEn = pd.read_excel('scimagojr-3.xlsx', na_values = np.NaN)
    top = answer_one()
    ContinentDict  = {'China':'Asia', 
                      'United States':'North America', 
                      'Japan':'Asia', 
                      'United Kingdom':'Europe', 
                      'Russian Federation':'Europe', 
                      'Canada':'North America', 
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}
    merged = pd.merge(top, ScimEn, how='inner', left_on='Rank', right_on = 'Rank')
    merged['cont'] = [ContinentDict[c] for c in merged['Country']]
    labels = [0,1,2,3,4]
    merged['bins'] = pd.cut(merged['% Renewable'], bins=5,labels = labels)
    cur = merged[['cont', 'bins']]
    grouped = cur.groupby(by=['cont', 'bins'])
    return grouped.size()

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [ ]:
def answer_thirteen():
    top = answer_one()
    top['pop'] = top['Energy Supply'] / top['Energy Supply per Capita']
    top['pop'] = ['{:,}'.format(r) for r in top['pop']]
    return top['pop']

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
# plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!